In [ ]:
import peekingduck
assert peekingduck.__version__ == 'v1.1.0' , "Peekingduck is not Updated to the latest version."

In [ ]:
from peekingduck.runner import Runner
from peekingduck.pipeline.nodes.input import live
from peekingduck.pipeline.nodes.model import yolo
from peekingduck.pipeline.nodes.draw import bbox, legend, blur_bbox
from peekingduck.pipeline.nodes.dabble import fps
from peekingduck.pipeline.nodes.output import media_writer, screen
from peekingduck.pipeline.nodes.node import AbstractNode
from custom_nodes.debug import printPipe
from custom_nodes.tracking import tracker
from custom_nodes.tracking import counter

In [ ]:
# input_node = recorded.Node(input_dir="t1.jpg", mirror_image=True)
# input_node = live.Node(input_source = "https://download.samplelib.com/mp4/sample-5s.mp4")
input_node = live.Node(
    threading = True
)
yolo_node = yolo.Node(
    detect_ids =[0,2], # detect human for 0, car for 2
    # yolo_iou_threshold = .2
    )
fps_node = fps.Node(fps_log_display=True)
draw_bbox_node = bbox.Node()
legend_node = legend.Node()
output_node = screen.Node()
tracker_node = tracker.Node()
# the 0.5 is just a random ROI value
counter_node = counter.Node(0.5)
blur_bbox_node = blur_bbox.Node()
print_node = printPipe.Node(dataInPipe = ['bboxes','trackers'], skipFrames = 30)

In [ ]:
runner = Runner(nodes=[
    input_node, 
    yolo_node, 
    tracker_node,
    print_node,
    counter_node,
    fps_node,
    draw_bbox_node,
    blur_bbox_node,
    legend_node,
    output_node])
runner.run()

In [ ]:
import cv2
cv2.destroyAllWindows()

In [ ]:
runner.pipeline.data.keys()